In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
tf.__version__

'2.15.0'

In [5]:
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(y)

[1 0 1 ... 1 1 0]


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [9]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [20]:
X[0]

array([1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
ann = tf.keras.models.Sequential()

In [15]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [16]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [17]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [18]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 4s 5ms/step - loss: 0.6158 - accuracy: 0.6609
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4622 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4395 - accuracy: 0.7969
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.8120
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4141 - accuracy: 0.8275
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3994 - accuracy: 0.8351
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3861 - accuracy: 0.8419
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3765 - accuracy: 0.8460
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3694 - accuracy: 0.8474
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3646 - accura

In [24]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1,50000]])) > 0.5)

1/1 [==============================] - 0s 20ms/step
[[False]]


In [25]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 1ms/step


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)

In [27]:
cm

array([[1526,   69],
       [ 204,  201]])

In [28]:
accuracy_score(y_test,y_pred)

0.8635